# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,2.12,96,99,2.60,GS,1699580105
1,1,puerto natales,-51.7236,-72.4875,9.25,49,40,6.17,CL,1699580110
2,2,umba,66.6871,34.3429,-4.17,91,92,1.74,RU,1699580676
3,3,gaalkacyo,6.7697,47.4308,24.19,74,98,5.29,SO,1699580676
4,4,malango,-2.9627,119.9001,28.48,54,19,1.76,ID,1699580677


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = .75,
    color = "City")

# Display the map
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
city_data_df['Wind Speed'].describe()

count    575.000000
mean       3.577983
std        2.531662
min        0.000000
25%        1.610000
50%        3.090000
75%        4.880000
max       13.300000
Name: Wind Speed, dtype: float64

In [5]:
# Narrow down cities that fit criteria and drop any results
ideal_weather= city_data_df[(city_data_df['Humidity'] <= 100) 
                            & (city_data_df['Cloudiness'] == 0) 
                            & (city_data_df['Wind Speed'] < 7)
                            &(city_data_df['Max Temp'] >21) 
                            & (city_data_df['Max Temp'] < 29)]

# Drop any rows with null values
ideal_weather.dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,wailua homesteads,22.0669,-159.3780,27.44,78,0,3.58,US,1699580137
57,57,masjed soleyman,31.9364,49.3039,21.62,37,0,0.57,IR,1699580698
111,111,tura,25.5198,90.2201,21.27,69,0,1.10,IN,1699580144
242,242,alamos,27.0167,-108.9333,22.67,51,0,1.37,MX,1699580781
298,298,petrolina,-9.3986,-40.5008,26.03,69,0,3.09,BR,1699580565
301,301,bhagalpur,25.2500,87.0000,21.66,53,0,1.25,IN,1699580809
341,341,saint-pierre,-21.3393,55.4781,22.82,68,0,2.06,RE,1699580829
412,412,akjoujt,19.6969,-14.4164,22.68,24,0,4.18,MR,1699580862
429,429,dwarka,22.2394,68.9678,26.87,69,0,4.12,IN,1699580872
444,444,sao francisco,-15.9486,-44.8644,23.63,39,0,1.79,BR,1699580878


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= ideal_weather[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']= ''

# Display sample data
hotel_df

C:\Users\sbelz\AppData\Local\Temp\ipykernel_7308\2890028000.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']= ''


,City,Country,Lat,Lng,Humidity,Hotel Name
18,wailua homesteads,US,22.0669,-159.3780,78,
57,masjed soleyman,IR,31.9364,49.3039,37,
111,tura,IN,25.5198,90.2201,69,
242,alamos,MX,27.0167,-108.9333,51,
298,petrolina,BR,-9.3986,-40.5008,69,
301,bhagalpur,IN,25.2500,87.0000,53,
341,saint-pierre,RE,-21.3393,55.4781,68,
412,akjoujt,MR,19.6969,-14.4164,24,
429,dwarka,IN,22.2394,68.9678,69,
444,sao francisco,BR,-15.9486,-44.8644,39,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
geoapify_key

'15683e611960489392461c308cede62b'

In [11]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit': 15
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row ['Lat']
    long = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"]=f'circle:{long},{lat},{radius}'
    params["bias"]=f'proximity:{long},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
masjed soleyman - nearest hotel: هتل جهانگردی
tura - nearest hotel: No hotel found
alamos - nearest hotel: No hotel found
petrolina - nearest hotel: Neuman
bhagalpur - nearest hotel: Hotel Shiva
saint-pierre - nearest hotel: Tropic Hotel
akjoujt - nearest hotel: فندق فاطمة
dwarka - nearest hotel: The Dwarika Hotel
sao francisco - nearest hotel: Hotel Atalaia
remire-montjoly - nearest hotel: Complexe Belova
key west - nearest hotel: Harborside Hotel & Marina
cozumel - nearest hotel: Hotel Isla Esmeralda
sainte-rose - nearest hotel: Habitation du Compté
prado - nearest hotel: Pousada Recanto do Prado


,City,Country,Lat,Lng,Humidity,Hotel Name
18,wailua homesteads,US,22.0669,-159.3780,78,Hilton Garden Inn Kauai Wailua Bay
57,masjed soleyman,IR,31.9364,49.3039,37,هتل جهانگردی
111,tura,IN,25.5198,90.2201,69,No hotel found
242,alamos,MX,27.0167,-108.9333,51,No hotel found
298,petrolina,BR,-9.3986,-40.5008,69,Neuman
301,bhagalpur,IN,25.2500,87.0000,53,Hotel Shiva
341,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
412,akjoujt,MR,19.6969,-14.4164,24,فندق فاطمة
429,dwarka,IN,22.2394,68.9678,69,The Dwarika Hotel
444,sao francisco,BR,-15.9486,-44.8644,39,Hotel Atalaia


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot

hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = .75,
    color = "City",
    hover_cols= ['Hotel Name', 'Country'])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)